<a href="https://colab.research.google.com/github/unmatched78/just_doit/blob/main/ML_Regression_Quality_Prediction_in_a_Mining_Proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
edumagalhaes_quality_prediction_in_a_mining_process_path = kagglehub.dataset_download('edumagalhaes/quality-prediction-in-a-mining-process')

print('Data source import complete.')


In [ ]:
# done in kaggle
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ML_Regression_Quality Prediction in a Mining Process

![](https://d12oja0ew7x0i8.cloudfront.net/image-handler/ts/20191016081931/ri/950/src/images/Article_Images/ImageForArticle_18547(1).jpg)

##### DataSource: <https://www.kaggle.com/datasets/edumagalhaes/quality-prediction-in-a-mining-process>


### Project Objective: To predict % silica in product after flotation process. The expected outcome for this project are as follows:
1. Improve reliability of process
2. Reduce waste water from process leading to environment friendly
3. Reduce cost of laboratory testing as soft sensor

### Major Conclusion:
1. Selected the random forest model as the primary model for deployment
2. the performance of selected model as belows:
    - Root Mean Square Error: 0.185
    - R2: 0.853
    

In [ ]:
# Import essential library

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split

## Load & Preview Data

### Finding & Action

- Drop date column
- Transform , to . and object type from str to float
- Specific Label as % silica concentration
- Not found missing value


In [ ]:
# load data to Dataframe

data=pd.read_csv('/kaggle/input/quality-prediction-in-a-mining-process/MiningProcess_Flotation_Plant_Database.csv')

data.head()

In [ ]:
data.info()

In [ ]:
# Drop date column
data_cl=data.drop('date',axis=1)
data_cl.head(5)

In [ ]:
# Transform , to . and object type from str to float

for dt in data_cl.columns:
    data_cl[dt]=data_cl[dt].str.replace(',','.').astype(float)

In [ ]:
data_cl.head(5)

In [ ]:
data_cl.info() #precleaning data

## Data Exploration

### > Data Correlation

In [ ]:
plt.figure(figsize=(20,20))
sn.heatmap(data_cl.corr(), annot=True)
plt.show()

#### From Correlation Matrix, possible features relating with label are as follows:

- iron concentrate
- amina flow
- ore pulp flow
- ore pulp density
- flotation air flow (01|02|03)
- flotation column level (04|05|06|07)
- % silica feed (**Added by sense of my mind)


In [ ]:
data_cl.columns

In [ ]:
## Feature vs Label Correlation Plot

fig, axs =plt.subplots(4,2,figsize=(8, 8))
fig.suptitle('Possible features correlation to label')


axs[0,0].scatter(data_cl['% Iron Concentrate'],data_cl['% Silica Concentrate'])
axs[0,0].set_xlabel(['% Iron Concentrate'])
axs[0,0].set_ylabel(['% Silica Concentrate'])

axs[0,1].scatter(data_cl['Amina Flow'],data_cl['% Silica Concentrate'])
axs[0,1].set_xlabel(['Amina Flow'])
axs[0,1].set_ylabel(['% Silica Concentrate'])

axs[1,0].scatter(data_cl['Ore Pulp Flow'],data_cl['% Silica Concentrate'])
axs[1,0].set_xlabel(['Ore Pulp Flow'])
axs[1,0].set_ylabel(['% Silica Concentrate'])

axs[1,1].scatter(data_cl['Ore Pulp Density'],data_cl['% Silica Concentrate'])
axs[1,1].set_xlabel(['Ore Pulp Density'])
axs[1,1].set_ylabel(['% Silica Concentrate'])

axs[2,0].scatter(data_cl['Flotation Column 01 Air Flow'],data_cl['% Silica Concentrate'])
axs[2,0].set_xlabel(['Flotation Column 01 Air Flow'])
axs[2,0].set_ylabel(['% Silica Concentrate'])

axs[2,1].scatter(data_cl[ 'Flotation Column 04 Level'],data_cl['% Silica Concentrate'])
axs[2,1].set_xlabel([ 'Flotation Column 04 Level'])
axs[2,1].set_ylabel(['% Silica Concentrate'])


axs[3,0].scatter(data_cl['% Silica Feed'],data_cl['% Silica Concentrate'])
axs[3,0].set_xlabel(['% Silica Feed'])
axs[3,0].set_ylabel(['% Silica Concentrate'])


axs[3,1].scatter(data_cl['% Silica Concentrate'],data_cl['% Silica Feed'])
axs[3,1].set_xlabel(['% Silica Concentrate'])
axs[3,1].set_ylabel(['% Silica Feed'])


plt.tight_layout()


### >> Data exploration Conclusion

##### The significant feature relating with label is on '% iron concentrate' in the product so it is not proper if the model is built without % iron concentrate as feature

Action:

- Select only the possible features, drop other column
- Do feature engineering for % iron concentrate (from x to x^2)

In [ ]:
# Action1: Select only the possible features, drop other column

data_cl.columns
droplist=['% Iron Feed','Starch Flow',\
          'Ore Pulp pH','Flotation Column 02 Air Flow',\
          'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',\
          'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',\
          'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
          'Flotation Column 02 Level', 'Flotation Column 03 Level', 'Flotation Column 05 Level',
          'Flotation Column 06 Level', 'Flotation Column 07 Level',]

data_cl=data_cl.drop(droplist,axis=1)
data_cl.shape

In [ ]:
# Action2 : Polynomial Feature Engineering for

data_cl['% Iron Concentrate_power2']=data_cl['% Iron Concentrate']**2

data_cl


## Training & Selection Regression Model

- Linear Regression Model with Regularization Term (Lassso Model)
- Linear Regression Model with Regularization Term (Ridge Model)
- Random Forest Model


In [ ]:
data_cl.columns

In [ ]:
## Split data to training set and test set

from sklearn.model_selection import train_test_split

X=data_cl.drop('% Silica Concentrate',axis=1)

y=data_cl['% Silica Concentrate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(y.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# normalize the training data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)

print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_train_norm,axis=0)}")


In [ ]:
# Define Validation Function

from sklearn.model_selection import cross_validate

def model_trainval(model,x,y):

    scoring = ['neg_root_mean_squared_error', 'r2']
    scores = cross_validate(model, x, y, scoring=scoring)
    RMSE = scores['test_neg_root_mean_squared_error'].mean()
    R2 = scores['test_r2'].mean()
    print("Result of Model Validation")
    print(f"RMSE : {RMSE}")
    print(f"R2 :{R2}")

    return RMSE,R2


In [ ]:
### Linear Regression Model with Regularization Term (Lassso)

from sklearn.linear_model import Lasso

reg1=Lasso(alpha=0.001)

## Model Training & Cross Validation

[RMSE, R2]=model_trainval(reg1,X_train_norm,y_train)

RMSE_Col=[]
R2_Col=[]

RMSE_Col.append(RMSE)
R2_Col.append(R2)


In [ ]:
### Linear Regression Model with Regularization Term (Ridge Model)

from sklearn.linear_model import Ridge

reg2=Ridge(alpha=0.001)

## Model Training & Cross Validation

[RMSE,R2]=model_trainval(reg2,X_train_norm,y_train)

RMSE_Col.append(RMSE)
R2_Col.append(R2)

In [ ]:
### Random Forest Tree Model

from sklearn.ensemble import RandomForestRegressor

reg3=RandomForestRegressor(max_depth=10,n_estimators=10)

## Model Training & Cross Validation

[RMSE,R2]=model_trainval(reg3,X_train_norm,y_train)

RMSE_Col.append(RMSE)
R2_Col.append(R2)

##### Remark: I try to reduce n_estimators to reduce time computation. At n_estimators=100 and n_estimators=10 are not different significantly so I decide to manually adjust n_estimators =10 to reduce training time  

In [ ]:
# Tree graphic model

reg3.fit(X_train_norm,y_train)

plt.figure(figsize=(50,50))

from sklearn.tree import plot_tree

plot_tree(reg3.estimators_[0],filled=True)


##### Remark: The trees seem to be complicated so we will test it again with test data to monitor overshooting

In [ ]:
print(RMSE_Col,R2_Col)

In [ ]:
# plot graph performance comparison each pre-selected model

Xgraph=['Losso','Ridge','RandomForest']

fig,axes=plt.subplots(nrows=1,ncols=2)

fig.suptitle("Scoring Result")
axes[0].bar(Xgraph,RMSE_Col)
axes[0].set_xlabel("Model")
axes[0].set_ylabel("RMSE")

axes[1].bar(Xgraph,R2_Col,color='g')
axes[1].set_xlabel("Model")
axes[1].set_ylabel("R2")

plt.tight_layout()
plt.show

### Result: a Random Forest model is selected as per scoring result

## Model Testing (before deployment)


In [ ]:
# normalize the test data

scaler = StandardScaler()
X_test_norm = scaler.fit_transform(X_test)


## Prediction
y_pred=reg3.predict(X_test_norm)

## Evaluation

from sklearn.metrics import mean_squared_error, r2_score

RMSE_Testing=mean_squared_error(y_test,y_pred)
R2_Testing=r2_score(y_test,y_pred)

print(f"RMSE of test result : {RMSE_Testing} ")
print(f"R2 of test result : {R2_Testing} ")


#### Remark: The result shows that R2 on test data (R2=0.8539 ) and R2 on CV data (R2=0.8672) is not difference significantly so the the model should be just fit but It is monitored again during on production

In [ ]:
#Built Dataframe for test result

result_label=pd.DataFrame()

result_label['% silica concentrate']=y_test
result_label['predicted % silica concentrate']=y_pred

## Project Summarization

- The Random Forest Model is selected as the prediction model with Root Mean Square error at 0.181 wih R Square at     0.857

- Comparing performance between Cross-Validation data set and test set. we found the perfromance that it is not drop     significantly, so the model should be used proper in deployment step

- The % iron concentrate is still be used as feature because other features are not relate with the lable significantly, so we are still had to sample to check lab result.

- Compared the predicted % silica concentration with test set as below table;  

In [ ]:
print(result_label.reset_index().head(50))

using LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [ ]:
data['date'] = pd.to_datetime(data['date'])

# Select features
features = ['% Iron Feed', 'Starch Flow', 'Amina Flow', 'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density']
X = data[features]
y = data['% Silica Concentrate']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
scaler = MinMaxScaler()
# Replace commas with periods and convert to float
X_train.replace(',', '.', regex=True, inplace=True)

# Now you can convert the entire dataframe to float
X_train = X_train.astype(float)

# Scale the data
X_train_scaled = scaler.fit_transform(X_train)

#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)
X_test.replace(',', '.', regex=True, inplace=True)

# Convert the entire dataframe to float
X_test = X_test.astype(float)

# Scale the data
X_test_scaled = scaler.transform(X_test)

# Reshape data for LSTM
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
y_train = y_train.str.replace(',', '.').astype(float)
y_test = y_test.str.replace(',', '.').astype(float)

history = model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test), verbose=2, shuffle=False)